# Detector de fumadores

<img src="../img/fumadores.jpg" alt="Personas fumando" width="50%" />


En este notebook y en los siguientes se va a desarrollar desde cero el proyecto de machine learning en el que gracias a los datos recogidos y ya anonimizados de miles de pacientes encuestados y estudiados en chequeos básicos se entrenará un modelo de predicción sobre el estado de salud y de los valores de los diferentes biomarcadores de estos. 

Esto ayudará a la detección precoz de posibles enfermedades derivadas por el consumo de tabaco.

Crear un modelo que detecte si un paciente es fumador o no es importante por varias razones. Ayuda a los profesionales de la salud a identificar a los individuos en riesgo de enfermedades relacionadas con el tabaco, facilita la implementación de programas de cesación del tabaquismo personalizados, y mejora la precisión del diagnóstico y tratamiento de condiciones médicas influenciadas por el hábito de fumar. Además, puede contribuir a la investigación médica al proporcionar datos valiosos sobre la relación entre el tabaquismo y diversas condiciones de salud.

Gracias a este proceso de entrenamiento será una forma de doble validación en caso de que el paciente no diga toda la verdad en cuanto a su consumo real de tabaco, detectar pacientes que sean fumadores pasivos de forma severa e incluso detectar precozmente posibles enfermedades derivadas por la contaminación ambiental aunque en los cuestionarios que se les haga a los pacientes respondan que no son fumadores.



## Antes de empezar

Si no lo has hecho ya, te recomendamos que visites el **[archivo README.md](../../README.md)** en el que se explica todo lo necesario para el correcto funcionamiento de todas las partes del proyecto.

## Procedencia de los datos

En esta ocasión, los datos que se han recolectado provienen de chequeos básicosrealizado en múltiples puntos de atención sanitaria, realizados por personal sanitario. A continuación se muestra lo que incluyen y valores de referencia generales para que se entienda que tipo de datos se observarán de aqui en adelante:

| Biomarcador | Unidad de Medida | Rango Normal (población sana) | Descripción |
|-------------|------------------|-------------------------------|-------------|
| Edad | Años | 20 a 85 | La edad del individuo. |
| Altura | cm | Varía por género y población | Medida de la estatura de una persona. |
| Peso | kg | Índice de Masa Corporal (IMC) adecuado para altura | Masa corporal total de una persona. |
| Circunferencia cintura | cm | Varía por género y población | Medición alrededor del área del abdomen. |
| Agudeza visual (izquierda/derecha) | - | 20/20 sin corrección visual | Capacidad para ver y distinguir detalles finos. |
| Audición (izquierda/derecha) | - | Sin pérdida auditiva | Capacidad para oír sonidos adecuadamente. |
| Presión arterial (sistólica/diastólica) | mm Hg | 90-120/60-80 | Fuerza que ejerce la sangre contra las paredes de las arterias. |
| Azúcar en sangre en ayuno | mg/dL | <100 | Nivel de glucosa en sangre en ayunas. |
| Colesterol total | mg/dL | <200 | Medida del colesterol en sangre. |
| Triglicéridos | mg/dL | <150 | Tipo de grasa (lípido) en la sangre. |
| HDL (colesterol bueno) | mg/dL | >60 | Colesterol de alta densidad, asociado con menor riesgo de enfermedad cardíaca. |
| LDL (colesterol malo) | mg/dL | <100 | Colesterol de baja densidad, asociado con mayor riesgo de enfermedad cardíaca. |
| Hemoglobina | g/dL | Varía por género y edad | Proteína en los glóbulos rojos que transporta oxígeno. |
| Proteína en orina | - | Ausente | Presencia de proteínas en la orina, puede indicar daño renal. |
| Creatinina sérica | mg/dL | 0.7-1.3 (hombres), 0.6-1.1 (mujeres) | Producto de desecho de la creatina, mide la función renal. |
| AST | U/L | 10-40 | Enzima asociada con el hígado y otras partes del cuerpo. |
| ALT | U/L | 7-56 | Enzima que ayuda al hígado a convertir alimentos en energía. |
| GTP (γ-GTP) | U/L | 9-48 | Enzima asociada con el hígado, puede indicar daño hepático. |
| Caries dental | Booleano (Verdadero/Falso) | - | Presencia o ausencia de caries en los dientes. |
| Fumador | Booleano (Verdadero/Falso) | - | Indica si la persona fuma tabaco. |


Por supuesto, conforme avance el proceso de creación del modelo se observará si son necesarios estos datos o únicamente algunos de ellos.

## Carga de datos

En total tenemos dos datasets ya divididos para la finalidad del proyecto en train, para entrenar el modelo con muchos más datos y en test, para probar la efectividad de este con menos datos.

La diferencia entre ambos es que train contiene la información sobre si son fumadores o no, mientras que el dataset test, la omite.

En las siguientes celdas cargamos ambos:

In [5]:
import pandas as pd

train = pd.read_csv('../data/raw/train.csv')
print('Archivo cargado')
train.head()

Archivo cargado


,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1


In [4]:
test = pd.read_csv('../data/raw/test.csv')
print('Archivo cargado')
test.head()

Archivo cargado


,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,159256,40,165,70,84.0,1.2,1.2,1,1,130,...,186,49,115,14.2,1,0.9,19,25,32,0
1,159257,80,160,60,93.0,1.0,1.0,2,2,144,...,158,35,104,13.0,1,1.1,20,12,24,0
2,159258,60,170,70,86.5,0.6,0.7,1,1,117,...,173,39,88,15.4,1,1.4,38,60,36,0
3,159259,40,160,50,67.0,0.3,0.4,1,1,116,...,47,75,128,14.5,1,0.6,25,18,10,1
4,159260,40,170,75,89.4,1.0,0.9,1,1,132,...,100,39,123,16.5,1,1.0,30,39,27,1


Una vez cargados los archivos vemos rápidamente que cantidad y tipo de datos estamos manejando.

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   159256 non-null  int64  
 1   age                  159256 non-null  int64  
 2   height(cm)           159256 non-null  int64  
 3   weight(kg)           159256 non-null  int64  
 4   waist(cm)            159256 non-null  float64
 5   eyesight(left)       159256 non-null  float64
 6   eyesight(right)      159256 non-null  float64
 7   hearing(left)        159256 non-null  int64  
 8   hearing(right)       159256 non-null  int64  
 9   systolic             159256 non-null  int64  
 10  relaxation           159256 non-null  int64  
 11  fasting blood sugar  159256 non-null  int64  
 12  Cholesterol          159256 non-null  int64  
 13  triglyceride         159256 non-null  int64  
 14  HDL                  159256 non-null  int64  
 15  LDL              

In [6]:
train.describe()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
count,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,...,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000
mean,79627.500000,44.306626,165.266929,67.143662,83.001990,1.005798,1.000989,1.023974,1.023421,122.503648,...,55.852684,114.607682,14.796965,1.074233,0.892764,25.516853,26.550296,36.216004,0.197996,0.437365
std,45973.391572,11.842286,8.818970,12.586198,8.957937,0.402113,0.392299,0.152969,0.151238,12.729315,...,13.964141,28.158931,1.431213,0.347856,0.179346,9.464882,17.753070,31.204643,0.398490,0.496063
min,0.000000,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,77.000000,...,9.000000,1.000000,4.900000,1.000000,0.100000,6.000000,1.000000,2.000000,0.000000,0.000000
25%,39813.750000,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,...,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000,0.000000
50%,79627.500000,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000,0.000000
75%,119441.250000,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,...,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,32.000000,44.000000,0.000000,1.000000
max,159255.000000,85.000000,190.000000,130.000000,127.000000,9.900000,9.900000,2.000000,2.000000,213.000000,...,136.000000,1860.000000,21.000000,6.000000,9.900000,778.000000,2914.000000,999.000000,1.000000,1.000000


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 106171 entries, 0 to 106170
Data columns (total 23 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   106171 non-null  int64  
 1   age                  106171 non-null  int64  
 2   height(cm)           106171 non-null  int64  
 3   weight(kg)           106171 non-null  int64  
 4   waist(cm)            106171 non-null  float64
 5   eyesight(left)       106171 non-null  float64
 6   eyesight(right)      106171 non-null  float64
 7   hearing(left)        106171 non-null  int64  
 8   hearing(right)       106171 non-null  int64  
 9   systolic             106171 non-null  int64  
 10  relaxation           106171 non-null  int64  
 11  fasting blood sugar  106171 non-null  int64  
 12  Cholesterol          106171 non-null  int64  
 13  triglyceride         106171 non-null  int64  
 14  HDL                  106171 non-null  int64  
 15  LDL              

In [11]:
test.describe()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
count,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,...,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000,106171.000000
mean,212341.000000,44.426538,165.221322,67.125618,82.999892,1.004776,0.999483,1.024216,1.023980,122.475403,...,127.406203,55.922286,114.597404,14.793354,1.072798,0.892463,25.559616,26.565427,36.086888,0.195458
std,30649.072052,11.897138,8.837065,12.586569,8.946584,0.397690,0.385752,0.153719,0.152988,12.765542,...,66.352488,14.000552,26.943401,1.434296,0.340818,0.179927,9.504442,17.944195,30.928138,0.396555
min,159256.000000,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,71.000000,...,8.000000,18.000000,1.000000,5.000000,1.000000,0.100000,6.000000,1.000000,2.000000,0.000000
25%,185798.500000,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,...,77.000000,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000
50%,212341.000000,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,114.000000,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000
75%,238883.500000,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,...,164.000000,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,33.000000,44.000000,0.000000
max,265426.000000,85.000000,190.000000,130.000000,127.700000,9.900000,9.900000,2.000000,2.000000,213.000000,...,548.000000,148.000000,1220.000000,21.100000,6.000000,9.900000,778.000000,2914.000000,999.000000,1.000000


In [14]:
# Colocamos los valores máximos y mínimos de cada columna de una forma más visual

describe_df = train.describe()

min_max_df = describe_df.loc[['min', 'max']].transpose()
min_max_df.columns = ['Min', 'Max']
min_max_df.reset_index(inplace=True)
min_max_df.rename(columns={'index': 'Feature'}, inplace=True)
min_max_df

,Feature,Min,Max
0,id,0.0,159255.0
1,age,20.0,85.0
2,height(cm),135.0,190.0
3,weight(kg),30.0,130.0
4,waist(cm),51.0,127.0
5,eyesight(left),0.1,9.9
6,eyesight(right),0.1,9.9
7,hearing(left),1.0,2.0
8,hearing(right),1.0,2.0
9,systolic,77.0,213.0


In [21]:
# Antes de empezar con el EDA vamos a eliminar la columna 'id' de ambos datasets ya que no aportará nada
# en el proceso de modelado y predicción

train = train.drop('id', axis=1)
train.to_csv('../data/processed/train_p.csv', index=False)

test = test.drop('id', axis=1)
test.to_csv('../data/processed/test_p.csv', index=False)

## Siguientes pasos

Una vez tenemos preparado el entorno de trabajo y comprobado que los archivos de datos cargan correctamente nos aseguramos que los datos con los que vamos a trabajar son correctos. 

Con esta primera exploración vemos que tenemos más de **250.000 entradas de datos, con 23 variables o biomarcadores** que nos ayudarán al modelado. No hay valores nulos por lo tanto no se requiere de una limpieza de datos o imputación de valores faltantes. Únicamente hemos descartado los datos sobre los id ya que estamos seguros que no nos servirán para entrenar al modelo.

En el **[siguiente notebook](EDA.ipynb)** nos centraremos en el **Análisis Exploratorio de Datos (EDA)** de una forma profunda que nos ayudará a destacar aquellos biomarcadores más relevantes para mejorar la usabilidad del modelo.

<div style="text-align: center;">
    <a href="EDA.ipynb">
        <button style="padding: 10px 20px; font-size: 16px; cursor: pointer;">Análisis Exploratorio de Datos</button>
    </a>
</div>